In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

In [2]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 17.8 MB/s eta 0:00:00


In [3]:
def search_youtube(song_name, singer_name):
    """
    Searches YouTube for a song by name and singer, and returns the URL of the top result.

    Parameters:
    - song_name (str): The name of the song.
    - singer_name (str): The name of the singer.

    Returns:
    - str: The URL of the top YouTube search result.
    """
    query = f"{song_name} {singer_name}"
    command = f'yt-dlp "ytsearch:{query}" --get-id --skip-download'

    try:
        # Execute the search and retrieve video ID
        video_id = subprocess.check_output(command, shell=True, text=True).strip()
        youtube_url = f"https://www.youtube.com/watch?v={video_id}"
        return youtube_url
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during YouTube search: {e}")
        return None

In [4]:
import re
def download_youtube_audio(df_songs, output_folder, audio_format="mp3"):
    """
    Download audio from YouTube videos using URLs from a DataFrame and save them to a specified folder.

    Parameters:
    - df_songs (pd.DataFrame): DataFrame containing songs and their corresponding URLs.
    - output_folder (str): The folder where the audio files will be saved.
    - audio_format (str): The audio format for the output files (default is 'mp3').
    """
    # Create the output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Check if the 'url' column is present in the DataFrame
    if 'url' not in df_songs.columns:
        print("The 'url' column is missing from the dataset.")
        return

    youtube_pattern = re.compile(r'(https?://)?(www\.)?(youtube\.com/watch\?v=|youtu\.be/)([a-zA-Z0-9_-]{11})')
    audio_counter = 1  # Initialize the audio file counter

    for index, row in df_songs.iterrows():
        url = row["url"].strip()
        song_title = row["Song"].strip().replace('/', '-')  # Replace any slashes to avoid path issues

        if youtube_pattern.match(url):
            # Create the command to fetch audio
            audio_file_path = f"{output_folder}/{audio_counter}.{song_title}.{audio_format}"
            command = f'yt-dlp -x --audio-format {audio_format} -o "{audio_file_path}" "{url}"'
            try:
                subprocess.run(command, shell=True, check=True)
                print(f"[{audio_counter}] Audio fetched successfully: {song_title} - {url}")
                audio_counter += 1  # Increment counter only for successful fetch
            except subprocess.CalledProcessError as e:
                print(f"[{audio_counter}] No audio fetched for: {song_title} - {url} - Error: {e}")
                # Do not increment audio_counter if fetch fails
        else:
            print(f"[{audio_counter}] No audio fetched for: {song_title} - {url} - Invalid URL")

In [5]:
from IPython.display import Audio, display
import os


def play_audio(file_path_or_url):
    """
    Play an MP3 audio file from a local path or a URL.

    Parameters:
    - file_path_or_url (str): Path to the MP3 file or a URL of the audio file.
    """
    # Check if the provided path is a local file and exists
    if os.path.isfile(file_path_or_url):
        print(f"Playing local file: {file_path_or_url}")
    else:
        print(f"Playing audio from URL: {file_path_or_url}")

    # Load and play the audio file
    audio = Audio(file_path_or_url, autoplay=True)
    display(audio)

In [6]:
import requests
from bs4 import BeautifulSoup
from collections import Counter

def get_top_song_emotions_for_dataset(df, emotion_keywords):
    """
    This function fetches a minimum of 1 and a maximum of 2 emotions per song based on keywords
    found in the song's Wikipedia page. If less than 1 emotion is found, it will fill with 'NO EMOTION'.
    If the page is not found, 'Page not found' will be set.
    """
    for index, row in df.iterrows():
        song_name = row['Song']  # Assumes there is a 'Song' column
        query = song_name.replace(" ", "_")
        url = f"https://en.wikipedia.org/wiki/{query}"

        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            paragraphs = soup.find_all("p")

            # Counter to store detected emotions and their frequency
            emotion_counter = Counter()

            # Process each paragraph to detect emotions
            for paragraph in paragraphs:
                text = paragraph.get_text().lower()  # Convert text to lowercase
                for keyword in emotion_keywords:
                    if keyword in text:
                        emotion_counter[keyword.capitalize()] += 1  # Count occurrences of each emotion

            # Select the top 2 emotions based on frequency (if available)
            top_emotions = [emotion for emotion, count in emotion_counter.most_common(2)]

            # Handle the case where no emotions were found
            if len(top_emotions) == 0:
                df.at[index, 'Emotion'] = "NO EMOTION"
            else:
                # If there's only one emotion, leave it as it is, else join top 2
                df.at[index, 'Emotion'] = ', '.join(top_emotions)
        else:
            # In case the Wikipedia page is not found
            df.at[index, 'Emotion'] = "Page not found"

    return df


In [3]:
import librosa

In [2]:
# extracting numeric features from the audio mp3 file:
# Function to extract audio features
def extract_audio_features(file_path, n_fft=1024, hop_length=256):
    y, sr = librosa.load(file_path, sr=None)  # Load audio with original sampling rate
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    magnitude, _ = librosa.magphase(D)
    original_length = len(y)
    feature_vector = magnitude.flatten()  # Flatten to a 1D array
    return feature_vector, sr, original_length


In [4]:
import soundfile as sf
def reconstruct_audio_from_features(feature_vector, sr, original_length, output_path, n_fft=512, hop_length=256, n_iter=32):
    # Reshape the feature vector back to the 2D spectrogram format
    num_freq_bins = n_fft // 2 + 1
    magnitude = feature_vector.reshape((num_freq_bins, -1))

    # Use Griffin-Lim to reconstruct the audio from the magnitude spectrogram
    audio_reconstructed = librosa.griffinlim(magnitude, hop_length=hop_length, n_iter=n_iter)

    # Adjust the length of the reconstructed audio to match the original
    audio_reconstructed = pad_or_trim_audio(audio_reconstructed, original_length)

    # Save the reconstructed audio as a .wav file
    sf.write(output_path, audio_reconstructed, sr)
    print(f"Reconstructed audio saved as {output_path}")

In [5]:
def pad_or_trim_audio(audio, original_length):
    if len(audio) < original_length:
        # Pad with zeros if the reconstructed audio is shorter
        padding_length = original_length - len(audio)
        audio = np.pad(audio, (0, padding_length), 'constant')
    else:
        # Trim the audio if it's longer than the original length
        audio = audio[:original_length]
    return audio